# concordance

> Functionality for concordance analysis.
- toc: false
- page-layout: full

In [ ]:
#| default_exp concordance

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
import time
import numpy as np
import polars as pl
import math
from fastcore.basics import patch
import msgspec
import hashlib

In [ ]:
#| export
from conc.corpus import Corpus
from conc.result import Result
from conc.plot import Plot
from conc.core import logger, PAGE_SIZE, EOF_TOKEN_STR, ERR_TOKEN_STR

In [ ]:
#| hide
from conc.core import set_logger_state

In [ ]:
#| hide
import os

In [ ]:
#| hide
source_path = f'{os.environ.get("HOME")}/data/'
save_path = f'{os.environ.get("HOME")}/data/conc-test-corpora/'

path_to_toy_corpus = f'{save_path}toy.corpus'
path_to_brown_corpus = f'{save_path}brown.corpus'
path_to_reuters_corpus = f'{save_path}reuters.corpus'
path_to_gardenparty_corpus = f'{save_path}garden-party.corpus'

In [ ]:
#| export
class Concordance:
	""" Class for concordancing. """
	def __init__(self,
			  corpus:Corpus # Corpus instance
			  ): 
		self.corpus = corpus


In [ ]:
#| exporti
@patch
def _get_concordance_sort(self:Concordance, 
						 token_positions: list[np.ndarray], # token index to get sort columns for
						 sort_columns: list, # sort columns to use
						 ignore_punctuation: bool = True # whether to use punctuation tokens for sorts or skip
						 ) -> tuple[np.ndarray, np.ndarray]: # token ids for first sort column and corresponding sort order
	""" Get the first sort column for a concordance. """

	start_time = time.time()
	index = 'orth_index'
	seq = np.array(token_positions[0]+sort_columns[0])
	sort_column_ids = self.corpus.get_tokens_by_index('orth_index')[seq]
	if ignore_punctuation:
		sort_column_ids_punct_mask = np.isin(sort_column_ids, self.corpus.punct_tokens)
		next_column = 1
		while np.any(sort_column_ids_punct_mask):
			sum = np.sum(sort_column_ids_punct_mask)
			logger.debug(f'Punct token correction for {sum} tokens, pass {next_column}')
			sort_column_ids[sort_column_ids_punct_mask] = self.corpus.get_tokens_by_index('orth_index')[np.array(token_positions[0]+sort_columns[0]+next_column)][sort_column_ids_punct_mask]
			next_column += 1
			sort_column_ids_punct_mask = np.isin(sort_column_ids, self.corpus.punct_tokens)
			if next_column > len(sort_columns):
				sum = np.sum(sort_column_ids_punct_mask)
				logger.warning(f'Concordance sort column retrieval reached maximum columns, sorts not found = {sum}')
				break

	sort_column_order = self.corpus.token_ids_to_sort_order(sort_column_ids)
	logger.info(f'Concordance sort column ({sort_column_ids.shape[0]}) retrieval time: {(time.time() - start_time):.5f} seconds')
	return sort_column_ids, sort_column_order


In [ ]:
#| hide
toy = Corpus().load(path_to_toy_corpus)
report_toy = Concordance(toy)

In [ ]:
# load the corpus
brown = Corpus('brown').load(path_to_brown_corpus)

In [ ]:
# instantiate the Concordance class
report_brown = Concordance(brown)

In [ ]:
#| hide
set_logger_state('verbose')
token_str = 'dog'
ignore_punctuation = False
token_sequence, index_id = brown.tokenize(token_str, simple_indexing=True)
token_positions = brown.get_token_positions(token_sequence, index_id)

sort_column_ids, sort_column_order = report_brown._get_concordance_sort(token_positions, [1, 2, 3], ignore_punctuation=ignore_punctuation)
print(sort_column_ids.shape, sort_column_order.shape, token_positions[0].shape)
print(sort_column_ids[:10])
print(brown.token_ids_to_tokens(sort_column_ids)[:10])
print(sort_column_order[:10])
set_logger_state('quiet')


2025-06-29 16:42:06 - INFO - _init_token_arrays - Created tokens_array in 0.140 seconds
2025-06-29 16:42:06 - INFO - _init_token_arrays - Created tokens_lookup in 0.009 seconds
2025-06-29 16:42:06 - INFO - _init_token_arrays - Created tokens_sort_order in 0.016 seconds
2025-06-29 16:42:06 - INFO - tokenize - Tokenization time: 0.17050 seconds
2025-06-29 16:42:06 - DEBUG - get_tokens_by_index - Getting tokens for index: lower_index
2025-06-29 16:42:06 - INFO - get_tokens_by_index - Got tokens for index lower_index with exclude_punctuation False in 0.037 seconds
2025-06-29 16:42:06 - INFO - get_token_positions - Token indexing (79) time: 0.10005 seconds
2025-06-29 16:42:06 - DEBUG - get_tokens_by_index - Getting tokens for index: orth_index
2025-06-29 16:42:06 - INFO - get_tokens_by_index - Got tokens for index orth_index with exclude_punctuation False in 0.009 seconds
2025-06-29 16:42:06 - INFO - _get_concordance_sort - Concordance sort column (79) retrieval time: 0.01236 seconds


(79,) (79,) (79,)
[29064 38309 33838 15829   795 47534  9374 42833 36136 21550]
['license' '.' 'owners' 'catchers' 'bites' 'wagons' 'in' "''" 'a' 'would']
[29512    41 36156  9357  6606 53962 25571    14  1578 55324]


In [ ]:
#| hide
# set_logger_state('verbose')
# token_str = 'dog'
# ignore_punctuation = False
# token_sequence, index_id = brown.tokenize(token_str, simple_indexing=True)
# token_positions = brown.get_token_positions(token_sequence, index_id)

# sort_column_ids, sort_column_order = report_brown._get_concordance_all_sort(token_positions, [1, 2, 3], ignore_punctuation=ignore_punctuation)
# print(sort_column_ids.shape, sort_column_order.shape, token_positions[0].shape)
# print(sort_column_ids[0][:10])
# print(brown.token_ids_to_tokens(sort_column_ids[0])[:10])
# print(sort_column_order[0][:10])
# set_logger_state('quiet')


In [ ]:
#| exporti
@patch
def _build_concordance_result_with_sort(self:Concordance,
						sequence_length: int, # length of the sequence to concordance
						concordance_result_df: pl.DataFrame, # polars DataFrame with initial concordance columns
						concordance_columns: list[pl.Series], # list of concordance columns - each column is a series of token ids
						concordance_range: range, # range of concordance columns to use - these are positions of concordance columns
						sort_columns: list, # list of sort columns for concordance
						ignore_punctuation:bool = False # whether to ignore punctuation tokens in sort columns
						) -> list: # concordance result df with token ids and sort columns
	""" Build sort columns from concordance columns. """

	# # add on sort columns
	# for i, sort_column in enumerate(sort_columns):
	# 	if sort_column not in concordance_range:
	# 		logger.error(f'Sort column {sort_column} not in concordance range {concordance_range}')
	# 		raise ValueError(f'Sort column {sort_column} not in concordance range {concordance_range}')
	# 	sort_column_index = concordance_range.index(sort_column)
	# 	concordance_columns.append(concordance_columns[sort_column_index].alias(f'sort{i}'))

	concordance_result_df = concordance_result_df.with_columns(concordance_columns)

	if ignore_punctuation:
		# get all positive integers from sort_columns
		sort_columns_positive = [col for col in sort_columns if col > 0]
		sort_columns_negative = [col for col in sort_columns if col < 0]
		if len(sort_columns_positive) > 0:
			right_tokens = np.array(concordance_result_df.select([f'{i}' for i in concordance_range if i > 0 and i not in list(range(sequence_length))]).to_numpy())
			right_tokens = right_tokens.T
			logger.debug(f"Context tokens collected: {right_tokens.shape}")
			right_tokens = np.where(np.isin(right_tokens, self.corpus.punct_tokens), 0, right_tokens)
			right_tokens = self.corpus._shift_zeroes_to_end(right_tokens)
			logger.debug(f"Right tokens shape: {right_tokens.shape}")
			for i, sort_column in enumerate(sort_columns_positive):
				logger.debug(f"Adding left sort column sort{sort_columns.index(sort_column)} based on column {sort_column}")
				concordance_result_df = concordance_result_df.with_columns(
					# because only have positive columns after sequence the first position in right_tokens is sequence_length - so need to adjust sort_column by sequence_length
					pl.Series(self.corpus.token_ids_to_sort_order(right_tokens[sort_column-sequence_length])).alias(f'sort{sort_columns.index(sort_column)}'),
					pl.Series(self.corpus.token_ids_to_tokens(right_tokens[sort_column-sequence_length])).alias(f'sort_debug_{sort_columns.index(sort_column)}')
				)
		if len(sort_columns_negative) > 0:
			left_tokens = np.array(concordance_result_df.select([f'{i}' for i in concordance_range if i < 0]).to_numpy())
			left_tokens = left_tokens.T
			logger.debug(f"Context tokens collected: {left_tokens.shape}")
			left_tokens = np.where(np.isin(left_tokens, self.corpus.punct_tokens), 0, left_tokens)
			left_tokens = self.corpus._shift_zeroes_to_start(left_tokens)
			logger.debug(f"Left tokens shape: {left_tokens.shape}")
			for i, sort_column in enumerate(sort_columns_negative):
				logger.debug(f"Adding left sort column sort{sort_columns.index(sort_column)} based on column {sort_column}")
				concordance_result_df = concordance_result_df.with_columns(
					# because only have negative columns before sequence first position - can use sort_column direct
					pl.Series(self.corpus.token_ids_to_sort_order(left_tokens[sort_column])).alias(f'sort{sort_columns.index(sort_column)}'),
					pl.Series(self.corpus.token_ids_to_tokens(left_tokens[sort_column])).alias(f'sort_debug_{sort_columns.index(sort_column)}')
				)

	return concordance_result_df


In [ ]:
#| export
@patch
def concordance(self: Concordance, 
				token_str: str, # token string to get concordance for 
				context_length:int = 5, # number of words to show on left and right of token string
				order:str='1R2R3R', # order of sort columns - one of 1L2L3L, 3L2L1L, 2L1L1R, 1L1R2R, 1R2R3R, LEFT, RIGHT
				page_size:int=PAGE_SIZE, # number of results to display per results page
				page_current:int=1, # current page of results
				show_all_columns:bool = False, # df with all columns or just essentials
				use_cache:bool = True, # retrieve the results from cache if available (currently ignored)
				ignore_punctuation:bool = False, # whether to ignore punctuation in the concordance sort (experimental - undergoing testing)
				) -> Result: # concordance report results
	""" Report concordance for a token string. """

	# DONE - reducing data retrieved to just the sort columns and then doing the concordance display separately here
	# DONE - speed up the sort so that does a partial sort (e.g. just one or two columns) to get position of the slice - then handle ordering with smaller slice of data
	# IDEA: potentially get sort columns until small enough result
	
	if order not in ['1L2L3L', '3L2L1L', '2L1L1R', '1L1R2R', '1R2R3R', 'LEFT', 'RIGHT']:
		raise ValueError(f'Invalid order: order must be one of: 1L2L3L, 3L2L1L, 2L1L1R, 1L1R2R, 1R2R3R, LEFT, RIGHT')
	
	if order == 'LEFT':
		order = '1L2L3L'
	elif order == 'RIGHT':
		order = '1R2R3R'

	token_sequence, index_id = self.corpus.tokenize(token_str, simple_indexing=True)

	start_time = time.time()
	sequence_len = len(token_sequence[0])
	concordance_range = range(-1 * context_length, context_length + sequence_len)
	positional_columns = [str(x) for x in concordance_range]

	index = 'orth_index'

	use_cache = False # forcing off for now

	cache_id = tuple(['concordance'] + list(token_sequence) + [order])

	if use_cache == True and cache_id in self.corpus.results_cache:
		logger.info('Using cached concordance results')
		positional_columns = self.corpus.results_cache[cache_id][0]
		concordance_df = self.corpus.results_cache[cache_id][1]
		total_count = self.corpus.results_cache[cache_id][2]
		total_docs = self.corpus.results_cache[cache_id][3]
		sort_columns = self.corpus.results_cache[cache_id][4]
	else:
		logger.info('Processing concordance results')
		token_positions = self.corpus.get_token_positions(token_sequence, index_id)

		if len(token_positions[0]) == 0:
			logger.info('No tokens found')
			return Result(type = 'concordance', df=pl.DataFrame(), title=f'Concordance for "{token_str}"', description=f'No matches', summary_data={}, formatted_data=[])

		if order == '1L2L3L':
			sort_columns = [-1,-2,-3]
		elif order == '3L2L1L':
			sort_columns = [-3,-2,-1]
		elif order == '2L1L1R':
			sort_columns = [-2,-1,sequence_len + 1 - 1]
		elif order == '1L1R2R':
			sort_columns = [-1,sequence_len + 1 - 1,sequence_len + 2 - 1]
		else:
			# i.e. 1R2R3R
			sort_columns = [sequence_len + 1 - 1,sequence_len + 2 - 1,sequence_len + 3 - 1]

		# getting first sort column here
		sort_column_ids, sort_column_order = self._get_concordance_sort(token_positions, sort_columns, ignore_punctuation=ignore_punctuation)
		
		concordance_df = pl.DataFrame([pl.Series(name='index', values=token_positions[0]), pl.Series(name='sort0', values=sort_column_order), pl.Series(name=str(sort_columns[0]), values=sort_column_ids)])
		concordance_df = concordance_df.sort('sort0')
		concordance_df = concordance_df.with_row_index('row')

		total_count = len(concordance_df)
		total_docs = len(np.unique(self.corpus.get_tokens_by_index('token2doc_index')[np.array(token_positions[0])])) # REFACTORED - was using old self.corpus.token2doc_index

		if use_cache == True:
			self.corpus.results_cache[cache_id] = [positional_columns, concordance_df, total_count, total_docs, sort_columns]

	# working out relevant slice to populate 
	resultset_start = page_size*(page_current-1)
	resultset_len = page_size
	resultset_end = min(resultset_start + resultset_len, len(concordance_df) - 1)
	
	start_order = concordance_df['sort0'][resultset_start]
	end_order = concordance_df['sort0'][resultset_end]
	start_order_pos = concordance_df.filter(pl.col("sort0") == start_order).head(1)['row'].item()
	end_order_pos = concordance_df.filter(pl.col("sort0") == end_order).tail(1)['row'].item()
	logger.info(f'Concordance resultset start: {resultset_start}, end: {resultset_end}, start order: {start_order}, end order: {end_order}, start order pos: {start_order_pos}, end order pos: {end_order_pos}')
	
	# populating a smaller chunk of the concordance report - as only need to retrieve/sort a subset
	concordance_result_df = concordance_df.slice(start_order_pos, end_order_pos - start_order_pos + 1)

	results_start_time = time.time()
	concordance_columns = []
	seq = concordance_result_df['index'].to_numpy()
	for pos in concordance_range:
		tokens = self.corpus.get_tokens_by_index(index)[np.array(seq+pos)] # REFACTORED - was using getattr call to get orth_index here
		concordance_columns.append(pl.Series(name=str(pos), values=tokens))
		if pos in sort_columns:
			column_name = 'sort'+str(sort_columns.index(pos))
			if column_name != 'sort0':
				concordance_columns.append(pl.Series(name=column_name, values=self.corpus.token_ids_to_sort_order(tokens)))

	concordance_result_df = self._build_concordance_result_with_sort(sequence_len, concordance_result_df, concordance_columns, concordance_range, sort_columns, ignore_punctuation)

	logger.info(f'Concordance results ({len(concordance_columns[0])}) retrieval time: {(time.time() - results_start_time):.5f} seconds')

	#offsets_arr = np.array(self.corpus.offsets,dtype=np.uint64) # FIX
	#document_ids = np.searchsorted(offsets_arr, concordance_result_df['index'], side = 'right') - 1 
	document_ids = self.corpus.get_tokens_by_index('token2doc_index')[np.array(concordance_result_df['index'])] # REFACTORED to remove offsets functionality
	concordance_result_df = concordance_result_df.with_columns(pl.Series(name="doc_id", values=document_ids))
	concordance_result_df = concordance_result_df.sort(['sort0','sort1','sort2'])
		
	# slicing this further to get only the required page of results and then populating with left, keyword, right strings
	logger.info(f'Concordance resultset slice: {start_order_pos} to {end_order_pos}, page size: {page_size}, slicing at {resultset_start - start_order_pos} for {page_size} rows')
	concordance_view_df = concordance_result_df.slice(resultset_start - start_order_pos, page_size)

	concordance_left = []
	concordance_right = []
	concordance_keyword = []

	for pos in positional_columns:
		if int(pos) < 0:
			concordance_left.append(self.corpus.token_ids_to_tokens(concordance_view_df[str(pos)].to_numpy()))
		elif int(pos) == 0 or int(pos) < sequence_len:
			concordance_keyword.append(self.corpus.token_ids_to_tokens(concordance_view_df[str(pos)].to_numpy()))
		else:
			concordance_right.append(self.corpus.token_ids_to_tokens(concordance_view_df[str(pos)].to_numpy()))

	concordance_left = [(' '.join(column)).split(EOF_TOKEN_STR)[-1] for column in np.array(concordance_left).T]
	concordance_keyword = [' '.join(column) for column in np.array(concordance_keyword).T]
	concordance_right = [(' '.join(column)).split(EOF_TOKEN_STR)[0] for column in np.array(concordance_right).T]

	concordance_view_df = concordance_view_df.with_columns(pl.Series(name='left', values=concordance_left), pl.Series(name='node', values=concordance_keyword), pl.Series(name='right', values=concordance_right))

	total_pages = math.ceil(total_count/page_size)
	summary_data = {'total_count': total_count, 'total_docs': total_docs, 'page': page_current, 'total_pages': total_pages}
	formatted_data = [f'Total Concordance Lines: {total_count}', f'Total Documents: {total_docs}', f'Showing {min(page_size, total_count)} lines', f'Page {page_current} of {total_pages}']

	if show_all_columns == False:
		concordance_view_df = concordance_view_df[['doc_id', 'left', 'node', 'right']]
	
	logger.info(f'Concordance report time: {(time.time() - start_time):.5f} seconds')

	return Result(type = 'concordance', df=concordance_view_df, title=f'Concordance for "{token_str}"', description=f'{self.corpus.name}, Context tokens: {context_length}, Order: {order}', summary_data=summary_data, formatted_data=formatted_data)


In [ ]:
#| hide
# no result
assert report_toy.concordance('dsahjhdsjhdsa', context_length=5).df.select(pl.len()).item() == 0

In [ ]:
#| hide
report_toy.concordance('the', context_length=5, ignore_punctuation= True, order='RIGHT').display()

Concordance for "the" 
 
 
 Toy Corpus, Context tokens: 5, Order: 1R2R3R 
 
 
 Doc Id 
 Left 
 Node 
 Right 
 
 
 
 
 5 
 
 The 
 cat is climbing a tree 
 
 
 3 
 
 The 
 cat is meowing . 
 
 
 1 
 
 The 
 cat sat on the mat 
 
 
 4 
 
 The 
 dog is barking . 
 
 
 6 
 
 The 
 dog is digging a hole 
 
 
 2 
 
 The 
 dog sat on the mat 
 
 
 1 
 The cat sat on 
 the 
 mat . 
 
 
 2 
 The dog sat on 
 the 
 mat . 
 
 
 
 
 
 Total Concordance Lines: 8 
 


 
 Total Documents: 6 
 


 
 Showing 8 lines 
 


 
 Page 1 of 1

In [ ]:
#| hide
set_logger_state('verbose')
print(brown.tokenize('mr.', simple_indexing=True))
report_brown.concordance('mr.', context_length = 20, order='1R2R3R').display()
set_logger_state('quiet')

2025-06-29 16:42:08 - INFO - tokenize - Tokenization time: 0.00021 seconds
2025-06-29 16:42:08 - INFO - tokenize - Tokenization time: 0.00009 seconds
2025-06-29 16:42:08 - INFO - concordance - Processing concordance results
2025-06-29 16:42:08 - DEBUG - get_tokens_by_index - Getting tokens for index: lower_index
2025-06-29 16:42:08 - INFO - get_tokens_by_index - Tokens for index lower_index with exclude_punctuation False already cached, returning cached result in 0.000 seconds
2025-06-29 16:42:08 - DEBUG - get_tokens_by_index - Getting tokens for index: lower_index
2025-06-29 16:42:08 - INFO - get_tokens_by_index - Tokens for index lower_index with exclude_punctuation False already cached, returning cached result in 0.000 seconds
2025-06-29 16:42:08 - INFO - get_token_positions - Token indexing (0) time: 0.07306 seconds
2025-06-29 16:42:08 - INFO - concordance - No tokens found


([(np.int64(0), np.int64(38309))], 66)


"Concordance for ""mr."""
No matches


In [ ]:
#| hide
reuters = Corpus().load(path_to_reuters_corpus)
conc_reuters = Concordance(reuters)

In [ ]:
#| hide
conc_reuters.concordance('the company said', context_length = 5, order='1R2R3R').display()

Concordance for "the company said" 
 
 
 Reuters Corpus, Context tokens: 5, Order: 1R2R3R 
 
 
 Doc Id 
 Left 
 Node 
 Right 
 
 
 
 
 2744 
 through a tender offer . 
 The company said 
 " The negotiations would determine 
 
 
 10501 
 1.25 dlrs a share . 
 The company said 
 " this could bring earnings 
 
 
 8353 
 of gold per ton . 
 The company said 
 & lt;Manitoba Mineral Resources Ltd 
 
 
 2186 
 . In a statement , 
 the company said 
 , " The SEC action 
 
 
 8898 
 Co > of Japan . 
 The company said 
 , " The discussions have 
 
 
 6379 
 In a brief statement , 
 the company said 
 , " We are studying 
 
 
 6221 
 special cost escrow accounts , 
 the company said 
 , adding , that there 
 
 
 4264 
 close in near future , 
 the company said 
 , adding it is prepared 
 
 
 6319 
 taxes . In addition , 
 the company said 
 , Georgia Power 's contracts 
 
 
 4664 
 the conversion of debentures . 
 The company said 
 , however , it expects 
 
 
 10302 
 against 1987 net income . 
 The company said 
 , however , that the 
 
 
 6464 
 the distribution of assets , 
 the company said 
 , it expects shareholders to 
 
 
 1364 
 of the public offering , 
 the company said 
 , it expects the secured 
 
 
 2911 
 part of the transaction , 
 the company said 
 , it granted IDC Acquisition 
 
 
 6660 
 While awaiting FDA approval , 
 the company said 
 , it is proceeding with 
 
 
 3545 
 is a preliminary estimate , 
 the company said 
 , it may be used 
 
 
 1702 
 . In the meantime , 
 the company said 
 , it plans today to 
 
 
 4788 
 Based on preliminary results , 
 the company said 
 , net income rose to 
 
 
 3595 
 the rights become exercisable , 
 the company said 
 , those held by shareholders 
 
 
 4889 
 group of First Delaware , 
 the company said 
 . 
 
 
 
 
 
 Total Concordance Lines: 1173 
 


 
 Total Documents: 911 
 


 
 Showing 20 lines 
 


 
 Page 1 of 59

In [ ]:
#| hide
conc_reuters.concordance('economy is', context_length = 5, order='1R2R3R', page_current = 1, ignore_punctuation = True).display()
#conc_reuters.concordance('economy is', context_length = 5, order='1R2R3R', page_current = 2).display()
#conc_reuters.concordance('economy is', context_length = 5, order='1R2R3R', page_current = 3).display()


Concordance for "economy is" 
 
 
 Reuters Corpus, Context tokens: 5, Order: 1R2R3R 
 
 
 Doc Id 
 Left 
 Node 
 Right 
 
 
 
 
 7129 
 this as indication that the 
 economy is 
 barrelling along . " " 
 
 
 5696 
 Canada 's resource - based 
 economy is 
 being hurt by international market 
 
 
 9554 
 wise leadership here and the 
 economy is 
 being very well managed , 
 
 
 2245 
 of Japan economists agreed the 
 economy is 
 better placed now to cope 
 
 
 8065 
 debt , because the Brazilian 
 economy is 
 essentially healthy . He said 
 
 
 385 
 NEW DEBT FEARS The global 
 economy is 
 expected to weaken this year 
 
 
 1847 
 think M-1 shows that the 
 economy is 
 falling apart and the Fed 
 
 
 2243 
 . Lawson said the U.K. 
 Economy is 
 fundamentally sound and added that 
 
 
 8338 
 as an indication that the 
 economy is 
 generating any momentum and are 
 
 
 7388 
 stopped deteriorating or if the 
 economy is 
 heading for a recovery . 
 
 
 3182 
 , PRESIDENT SAYS The Kenyan 
 economy is 
 heading for difficult times after 
 
 
 7388 
 survey showed that the Japanese 
 economy is 
 in a delicate stage in 
 
 
 5026 
 pct generally indicates that the 
 economy is 
 in an expanding phase . 
 
 
 1513 
 a further stimulation of the 
 economy is 
 inappropriate , government spokesman Friedhelm 
 
 
 5993 
 coincides with signs that the 
 economy is 
 losing momentum as the quarter 
 
 
 6039 
 Benderley said . " The 
 economy is 
 moving between extremes , " 
 
 
 10243 
 dollar ) . The U.S. 
 Economy is 
 not good , and as 
 
 
 10289 
 dollar ) . The U.S. 
 Economy is 
 not good , and as 
 
 
 10243 
 as long as the U.S. 
 Economy is 
 not good , the U.S. 
 
 
 10289 
 as long as the U.S. 
 Economy is 
 not good , the U.S. 
 
 
 
 
 
 Total Concordance Lines: 43 
 


 
 Total Documents: 38 
 


 
 Showing 20 lines 
 


 
 Page 1 of 3

In [ ]:
report_brown.concordance('good at', context_length = 10, order='1R2R3R').display()

Concordance for "good at" 
 
 
 Brown Corpus, Context tokens: 10, Order: 1R2R3R 
 
 
 Doc Id 
 Left 
 Node 
 Right 
 
 
 
 
 484 
 about twenty miles away , and he was also pretty 
 good at 
 anything in the carpentry line . was a vivid , 
 
 
 263 
 he says , ' as a storyteller and was precociously 
 good at 
 description , dialogue , and most of the other staples 
 
 
 479 
 and not a method of passing the day . was 
 good at 
 his job . probably was n't hard for him to 
 
 
 474 
 trying to flatter her vanity . You must have been 
 good at 
 history at school . did you go to school '' 
 
 
 82 
 enough of unequal merit , but all of them pretty 
 good at 
 that . consisted of a new arrangement of ` ` 
 
 
 474 
 Why not '' ? ? said . I 'm not 
 good at 
 that kind of thing '' . This afternoon let 's 
 
 
 
 
 
 Total Concordance Lines: 6 
 


 
 Total Documents: 5 
 


 
 Showing 6 lines 
 


 
 Page 1 of 1

In [ ]:
#| hide
report_brown.concordance('dog', context_length = 10, order='1R2R3R', ignore_punctuation=True, show_all_columns = False).display()

2025-06-29 16:42:09 - WARNING - _get_concordance_sort - Concordance sort column retrieval reached maximum columns, sorts not found = 0


Concordance for "dog" 
 
 
 Brown Corpus, Context tokens: 10, Order: 1R2R3R 
 
 
 Doc Id 
 Left 
 Node 
 Right 
 
 
 
 
 463 
 would n't raise a finger except to smooth his yellow 
 dog 
 's back . would be time enough , perhaps the 
 
 
 392 
 stand up on the back of my neck like a 
 dog 
 's that is going to get into a fight . 
 
 
 61 
 men you will find anywhere . even gave my little 
 dog 
 a biscuit . was very refreshing . Deep Peep Show 
 
 
 193 
 of shade cast by the boulder . was also a 
 dog 
 , a dingo dog . ribs showed , it was 
 
 
 473 
 . Why put such a high value on being top 
 dog 
 '' ? ? added . was coarse , almost insulting 
 
 
 285 
 continue the equipping of the B-52 wings with the Hound 
 Dog 
 air - to - surface missile . the coming fiscal 
 
 
 83 
 the only canvas with an ash can ( and foraging 
 dog 
 ) , although Sloan was a member of the famous 
 
 
 306 
 intrapulmonary features of various mammalian lungs other than in the 
 dog 
 and cat ( Miller , ' 13 ; ; ) 
 
 
 56 
 into a baseball program , introduced the ` ` hot 
 dog 
 '' and paved the way for creation of a catering 
 
 
 204 
 a pleasure to contemplate '' ; ; wagon has no 
 dog 
 , and therefore is ` ` devoid of interest '' 
 
 
 463 
 yellow dog onto him . He was never without that 
 dog 
 . And his eyes -- those miniature sundials of variegated 
 
 
 110 
 or horsemanship classes -- riding a horse and showing a 
 dog 
 are very similar ! ! Your aids are your attitude 
 
 
 496 
 from the church social last Saturday night , a savage 
 dog 
 attacked them and bit Mr. Morgan on the public square 
 
 
 110 
 ` and what have you left ? ? a nervous 
 dog 
 because you are livid with rage -- a sure sign 
 
 
 110 
 too seriously and had better stop ! ! a bored 
 dog 
 because you are more interested in something else -- maybe 
 
 
 9 
 policemen assigned as dog catchers and $ 15,000 to investigate 
 dog 
 bites . higher fees City Finance Director Richard J. McConnell 
 
 
 110 
 care ? ? they indifferent to the fact that the 
 dog 
 can easily pick up germs from the preceding patient ? 
 
 
 306 
 , and pig ; ; 2 , , by the 
 dog 
 , cat , and monkey ; ; 3 , , 
 
 
 306 
 be applied to man . is especially so if the 
 dog 
 , cat or monkey are to be used , in 
 
 
 9 
 's S.P.C.A. ; ; to pay six policemen assigned as 
 dog 
 catchers and $ 15,000 to investigate dog bites . higher 
 
 
 
 
 
 Total Concordance Lines: 79 
 


 
 Total Documents: 30 
 


 
 Showing 20 lines 
 


 
 Page 1 of 4

In [ ]:
#| hide
# THIS NEEDS TO BE COMMENTED OUT BEFORE CI AS NOT INCLUDED IN SAMPLE DATA
# congress = Corpus().load(f'{save_path}us-congressional-speeches-subset-100k.corpus')
# report_congress = Concordance(congress)
# congress._init_token_arrays()
# congress = Corpus().load(f'{save_path}us-congressional-speeches-subset-500k.corpus')
# report_congress = Concordance(congress)


In [ ]:
#| hide
# THIS NEEDS TO BE COMMENTED OUT BEFORE CI AS NOT INCLUDED IN SAMPLE DATA
# set_logger_state('verbose')
# report_congress.concordance('the', context_length = 10, order='1R2R3R', ignore_punctuation=True, show_all_columns = False).display()
# set_logger_state('quiet')

In [ ]:
#| hide
# THIS NEEDS TO BE COMMENTED OUT BEFORE CI AS NOT INCLUDED IN SAMPLE DATA
# congress.tokens.with_row_index('position').filter(pl.col('position') > 78773800).join(
#     congress.vocab.select(pl.col('token_id'), pl.col('token'), pl.col('is_punct'), pl.col('is_space')), left_on='orth_index', right_on='token_id', how='left', maintain_order='left').head(20).collect()

In [ ]:
#| hide
# %time report_congress.concordance('god', context_length = 5, order='1R2R3R').display()

In [ ]:
#| hide
# congress.tokenize('government', simple_indexing=True)

In [ ]:
#| exporti
@patch
def _get_concordance_plot_style(
	self: Concordance,
	default_font_size: int = 12, # default font size for the plot
	) -> str: # HTML styles markup
	""" Get the HTML styles for the concordance plot. """

	html_styles = '''<style>
	.conc-plot-wrapper {
	background: #fff;
	width:1000px;
	color: #000;
	font-family: 'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, 'Helvetica Neue', 'Fira Sans', 'Droid Sans', Arial, sans-serif;
	line-height: 1.1;
	margin: 20px 0 20px 0;
	}
	.conc-plot-wrapper h2 {
	color: #000;
	font-family: 'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, 'Helvetica Neue', 'Fira Sans', 'Droid Sans', Arial, sans-serif;
	text-align:center;
	font-weight: 600;
	line-height: 2;
	margin:0;
	padding:0;
	}
	.conc-plot-wrapper h3 {
	color: #000;
	font-family: 'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, 'Helvetica Neue', 'Fira Sans', 'Droid Sans', Arial, sans-serif;
	text-align:center;
	font-weight: 300;
	line-height: 1;
	margin:0;
	padding:0;
	}
	.conc-concordance-plot {
	}
	.conc-concordance-plot-summary {
	margin: 20px 40px 10px 160px;
	color: #000;
	font-family: 'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, 'Helvetica Neue', 'Fira Sans', 'Droid Sans', Arial, sans-serif;
	}
	.conc-concordance-plot-controls {
	height: 60px;
	margin: 0 40px 20px 40px;
	}
	.conc-concordance-plot-controls input[type="range"] {
	-webkit-appearance: none;
	width: 100%;
	height: 15px;
	background: #ccc;
	border-radius: 5px;
	outline: none;
	opacity: 0.7;
	transition: opacity .2s;
	}
	.conc-concordance-plot-controls label {
	font-family: 'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, 'Helvetica Neue', 'Fira Sans', 'Droid Sans', Arial, sans-serif;
	color: #000;
	display: block;
	margin-bottom: 5px;
	}
	.conc-concordance-plot rect {
	fill: #ccc;
	width: 800px;
	height: 40px;
	}
	.conc-concordance-plot .label {
	font-family: 'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, 'Helvetica Neue', 'Fira Sans', 'Droid Sans', Arial, sans-serif;
	}
	g.conc-concordance-plot-line {
	cursor: pointer;
	}
	g.conc-concordance-plot-line line {
	stroke:#666;
	}
	g.conc-concordance-plot-line:hover line {
	stroke:#000;
	}
	g.conc-concordance-plot-line > text {
	font-family: 'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, 'Helvetica Neue', 'Fira Sans', 'Droid Sans', Arial, sans-serif;
	display:none;
	}
	g.conc-concordance-plot-line:hover > text {
	display:block;
	}
	g.conc-concordance-plot-line.highlight line {
	}
	'''
	html_styles += '.conc-plot-wrapper { font-size: ' + str(default_font_size) + 'px; }'
	html_styles += '.conc-plot-wrapper h2 { font-size: ' + str(default_font_size * 2) + 'px; }'
	html_styles += '.conc-plot-wrapper h3, .conc-concordance-plot-summary { font-size: ' + str(default_font_size * 1.25) + 'px; }'
	html_styles += '.conc-concordance-plot-controls label { font-size: ' + str(default_font_size) + 'px; }'
	html_styles += '</style>'

	return html_styles


In [ ]:
#| exporti
@patch
def _get_concordance_plot_script(
	self: Concordance,
	plot_id: str = '' # ID to use for the plot elements
	) -> str:
	""" Get the JavaScript for the concordance plot. """

	html_script = '''
	function filter_token_ids(token_ids, plot_id) {
		const data = window[plot_id];
		var centre_index = Math.floor(token_ids.length / 2);
		var left_tokens = token_ids.slice(0, centre_index);
		var right_tokens = token_ids.slice(centre_index);
		if (left_tokens.includes(data['eof_token'])) {
			var eof_index_pos = left_tokens.lastIndexOf(data['eof_token']);
			if (eof_index_pos !== -1) {
				left_tokens = left_tokens.slice(eof_index_pos + 1);
			}
		}
		if (right_tokens.includes(data['eof_token'])) {
			var eof_index_pos = right_tokens.indexOf(data['eof_token']);
			if (eof_index_pos !== -1) {
				right_tokens = right_tokens.slice(0, eof_index_pos);
			}
		}
		token_ids = left_tokens.concat(right_tokens);
		return token_ids;
	}
	function token_ids_to_str(token_ids, plot_id) {
		const data = window[plot_id];
		if (token_ids.includes(data['eof_token'])) {
			token_ids = filter_token_ids(token_ids, plot_id);
		}
		const token_strs = [];
		for (let i = 0; i < token_ids.length; i++) {
			const token_id = token_ids[i];
			if (data['tokens'][token_id]) { 
				token_strs.push(data['tokens'][token_id]); 
			} else {
				token_strs.push(`Unknown Token ID: ${token_id}`);
			}
		}
		return token_strs.join(' ');
	}
	function populatePlot(page, page_size, plot_id) {
		const data = window[plot_id];
		start = (page - 1) * page_size;
		end = start + page_size;
		const plot = document.getElementById(plot_id).querySelector('.conc-concordance-plot');
		const lines = plot.getElementsByClassName('conc-concordance-plot-line');
		while (lines.length > 0) {
			lines[0].remove();
		}
		const labels = plot.getElementsByClassName('label');
		while (labels.length > 0) {
			labels[0].remove();
		}
		const rects = plot.getElementsByTagName('rect');
		for (let i = 0; i < rects.length; i++) {
			rects[i].setAttribute('style', 'opacity:1;');
		}
		for (let i = start; i < end; i++) {
			page_i = (i - start + 1)
			if (i >= data['docs'].length) { 
				const rect = plot.querySelector(`.rect-${page_i}`);
				if (rect) {
					rect.setAttribute('style', 'opacity:0;');
				}
				continue;
			}
			const doc = data['docs'][i]; 
			const plot_y = (page_i * data['row_height']) - data['row_adjustment'];
			const label_y = plot_y + (data['default_font_size'] * 1.4);
			const label_y2 = plot_y + (data['default_font_size'] * 1.4 * 2);
			const doc_positions_count = doc.positions.length;
			const line_text = doc_positions_count === 1 ? '1 line' : `${doc_positions_count} lines`;
			const label = document.createElementNS('http://www.w3.org/2000/svg', 'text');
			label.setAttribute('class', 'label');
			label.setAttribute('x', data['label_x_right']);
			label.setAttribute('y', label_y);
			label.setAttribute('font-size', data['default_font_size']);
			label.setAttribute('text-anchor', 'end');
			label.textContent = `Doc ${doc.doc_id}`;
			plot.appendChild(label);
			const label2 = document.createElementNS('http://www.w3.org/2000/svg', 'text');
			label2.setAttribute('class', 'label');
			label2.setAttribute('x', data['label_x_right']);
			label2.setAttribute('y', label_y2);
			label2.setAttribute('font-size', data['default_font_size']);
			label2.setAttribute('text-anchor', 'end');
			label2.textContent = line_text;
			plot.appendChild(label2);
			const positions = doc.positions;
			const x_values = positions.map(pos => (pos / doc.count) * 100 * 8);
			for (let j = 0; j < x_values.length; j++) {
				const x_value = x_values[j];
				const line = document.createElementNS('http://www.w3.org/2000/svg', 'g');
				line.setAttribute('class', 'conc-concordance-plot-line');
				const line1 = document.createElementNS('http://www.w3.org/2000/svg', 'line');
				line1.setAttribute('x1', data['plot_x'] + x_value);
				line1.setAttribute('y1', plot_y);
				line1.setAttribute('x2', data['plot_x'] + x_value);
				line1.setAttribute('y2', plot_y + data['subplot_height']);
				line1.setAttribute('style', 'stroke-width:10;opacity:0;');
				line.appendChild(line1);
				const line2 = document.createElementNS('http://www.w3.org/2000/svg', 'line');
				line2.setAttribute('x1', data['plot_x'] + x_value);
				line2.setAttribute('y1', plot_y);
				line2.setAttribute('x2', data['plot_x'] + x_value);
				line2.setAttribute('y2', plot_y + data['subplot_height']);
				line2.setAttribute('style', 'stroke-width:2;');
				line.appendChild(line2);
				let anchor = 'middle';
				let x_adjustment = 0;
				if (x_value < 150) {
					anchor = 'start';
					x_adjustment = -30;
				}
				if (x_value > 650) {
					anchor = 'end';
					x_adjustment = 30;
				}
				const text = document.createElementNS('http://www.w3.org/2000/svg', 'text');
				text.setAttribute('x', data['plot_x'] + x_value + x_adjustment);
				text.setAttribute('y', plot_y - 5);
				text.setAttribute('text-anchor', anchor);
				text.setAttribute('font-size', '12');
				text.setAttribute('fill', 'black');
				append = ''
				if (data['append_info']) {
					position_offset_0 = doc.positions[j];
					position_offset_1 = doc.positions[j] + 1;
					append = ` (token ${position_offset_1} of ${doc.count})`;
					line.setAttribute('data-offset', `${position_offset_0}`);
				}
				text.textContent = token_ids_to_str(data['examples'][i].orth_indices[j], plot_id) + append;
				line.setAttribute('data-doc', `${doc.doc_id}`);
				line.appendChild(text);
				plot.appendChild(line);
			}

		}
	}

	function tryInit(plot_id) {
	const targetNode = document.getElementById(plot_id);
	if (targetNode && targetNode.querySelector('.conc-plot-footer')) {
		vizInit(plot_id);
		return true;
	}
	return false;
	}
	function vizInit(plot_id) {
		const data = window[plot_id];
		current_page = 1;
		const plot = document.getElementById(plot_id);
		const slider = plot.querySelector('.conc-concordance-plot-slider');
		populatePlot(current_page, data['page_size'], plot_id);
		slider.addEventListener('input', function() {
			const page = parseInt(this.value, 10);
			populatePlot(page, data['page_size'], plot_id);
			const page_number = plot.querySelector('.conc-concordance-plot-page-number');
			page_number.textContent = `${page}`;
		});
	}
	
	'''
	return html_script

In [ ]:
#| export
@patch
def concordance_plot(self: Concordance,
				token_str: str, # token string for concordance plot
				page_size: int = 10, # number of plots per page
				append_info: bool = True # append token position info to the concordance line preview screens visible when hover over the plot lines
				) -> Plot: # concordance plot object, add .display() to view in notebook
	"""Create a concordance plot."""

	# may make these configurable in function call in the future - but would need to ensure these don't have to passed to styles
	row_height = 60
	start_first_row_at = 30
	row_adjustment = row_height - start_first_row_at
	default_font_size = 12
	subplot_height = 40
	plot_height = start_first_row_at + (row_height * page_size) + subplot_height - row_height
	plot_x = 160
	label_x_right = plot_x - 10

	token_sequence, index_id = self.corpus.tokenize(token_str, simple_indexing=True)
	token_positions = self.corpus.get_token_positions(token_sequence, index_id)
	sequence_len = len(token_sequence[0])

	if len(token_positions[0]) == 0:
		print("No matches found.")
		return

	fn_string = f'{self.corpus.slug}{str(page_size)}{str(append_info)}{token_str}{time.strftime("%Y%m%d%H%M%S")}'
	plot_id = 'concplot_' + hashlib.md5(fn_string.encode('utf-8')).hexdigest()

	lines_df = self.corpus.tokens.with_row_index('position').filter(
		pl.col('position').is_in(token_positions[0])
	).select(pl.col('position'), pl.col('token2doc_index').alias('doc_id'), pl.col('orth_index').alias('orth_index_3')).sort(by = ['doc_id', 'position'])

	examples_df = lines_df

	lines_df = lines_df.join(
		self.corpus.tokens.with_row_index('position').group_by('token2doc_index').agg([
			pl.col('position').min().alias('min'),
			pl.col('position').max().alias('max')
		]).select(pl.col('token2doc_index').alias('doc_id'), pl.col('min'), pl.col('max')),
		on='doc_id',
		how='inner', #only want rows where doc_id is in lines_df
		maintain_order='left'
	)

	docs_df = lines_df.with_columns((pl.col('position') - pl.col('min')).alias('position'), (pl.col('max') - pl.col('min')).alias('count')).group_by('doc_id').agg([
		pl.col('position').alias('positions'),
		pl.col('count').first().alias('count'),
	])

	examples_df = examples_df.with_columns(
		(pl.col('position') - 3).alias('position_0'),
		(pl.col('position') - 2).alias('position_1'),
		(pl.col('position') - 1).alias('position_2'),
		(pl.col('position') + 1).alias('position_4'),
		(pl.col('position') + 2).alias('position_5'),
		(pl.col('position') + 3).alias('position_6'),
	)
	# using 3 tokens either side for initial release - see range below and hardcoded positions above - tweak so configurable in future release
	for i in range(0, 7): 
		if i == 3: # node
			continue
		else:
			examples_df = examples_df.join(
				self.corpus.tokens.with_row_index('position').select(
					pl.col('orth_index').alias(f'orth_index_{i}'),
					pl.col('position').alias(f'position_{i}')
				),
				on=f'position_{i}',
				how='inner',
				maintain_order='left'
			).drop(f'position_{i}')

	orth_index_columns = [f'orth_index_{i}' for i in range(7)]

	examples_df = examples_df.select(
		pl.col('doc_id'),
		pl.col('position'),
		*orth_index_columns
		).with_columns(pl.concat_list(orth_index_columns).alias('orth_indices')).sort(by=['doc_id', 'position'])

	unique_df = examples_df.explode('orth_indices').select(
		pl.col('orth_indices').unique().alias('token_id')
	)

	examples_df = examples_df.drop(orth_index_columns).drop('position')
	examples_df = examples_df.group_by('doc_id').agg(
		pl.concat_list(pl.col('orth_indices')).alias('orth_indices'),
	).sort(by='doc_id')
	
	unique_df = unique_df.join(
		self.corpus.vocab.select(pl.col('token_id'), pl.col('token')),
		on='token_id',
		how='left', 
		maintain_order='left'
	).sort(by='token_id')

	concordance_lines = len(token_positions[0])
	num_docs = docs_df.select(pl.len()).collect().item()
	num_pages = math.ceil(num_docs / page_size)

	html = ''
	# html = f'''<!doctype html>
	# <html lang="en">
	# <head>
	# 	<meta charset="UTF-8"><title>Conc Plot</title>
	# '''

	html += '<script>\n'
	html += f'var {plot_id}' + ' = {\n'
	html += f'eof_token: {self.corpus.EOF_TOKEN},\n'
	html += f'page_size: {page_size},\n'
	html += f'append_info: {("true" if append_info else "false")},\n'
	html += f'row_height: {row_height},\n'
	html += f'start_first_row_at: {start_first_row_at},\n'
	html += f'row_adjustment: {row_adjustment},\n'
	html += f'default_font_size: {default_font_size},\n'
	html += f'subplot_height: {subplot_height},\n'
	html += f'plot_height: {plot_height},\n'
	html += f'plot_x: {plot_x},\n'
	html += f'label_x_right: {label_x_right},\n'
	html += 'docs: ' + msgspec.json.encode([row for row in docs_df.collect().iter_rows(named=True)]).decode("utf-8") + ',\n'
	html += 'tokens: ' + msgspec.json.encode({row['token_id']: row['token'] for row in unique_df.collect().iter_rows(named=True)}).decode("utf-8") + ',\n'
	html += 'examples: ' + msgspec.json.encode([row for row in examples_df.collect().iter_rows(named=True)]).decode("utf-8") + '\n'
	html += '}\n\n'
	html += f'{(self._get_concordance_plot_script(plot_id))}\n'
	html += '</script>\n'
	html += self._get_concordance_plot_style(default_font_size=default_font_size)
	#html += '</head><body>'

	html += f'<div class="conc-plot-wrapper" id="{plot_id}">'
	html += f'<h2>Concordance Plot for &quot;{token_str}&quot;</h2>'
	html += f'<h3>{self.corpus.name}</h3>'
	html += f'<svg class="conc-concordance-plot" viewBox="0 0 1000 {plot_height}" width="1000" height="{plot_height}" xmlns="http://www.w3.org/2000/svg">'
	# tmp
	row_adjustment = row_height - start_first_row_at
	n_plots_this_page = page_size
	for i in range(1, page_size + 1):
		html += f'<rect class="rect-{i}" x="{plot_x}" y="{((i * row_height) - row_adjustment)}" height="40" width="800" />'
	html += '</svg>'
	html += f'<div class="conc-concordance-plot-summary">Total Documents: {num_docs}<br>Total Concordance Lines: {concordance_lines}</div>'
	html += f'''<div class="conc-concordance-plot-controls"><label for="{plot_id}-slider" class="conc-concordance-plot-slider-label">Page <span class="conc-concordance-plot-page-number">1</span> of {num_pages}</label>
	<input type="range" min="1" max="{num_pages}" value="1" step="1" class="slider conc-concordance-plot-slider" id="{plot_id}-slider" autocomplete="off"></div>
	<div class="conc-plot-footer"></div>
	</div>'''

	html += '<script>\n'
	html += 'if (!tryInit("' + plot_id + '")) {\n'
	html += '''
	// Only set up observer if not ready yet
	const observer = new MutationObserver(function(mutationsList, observer) {
	'''
	html += '''
		'''
	html += 'if (tryInit("' + plot_id + '")) {\n'
		
	html += '''
		observer.disconnect();
		}

	});
	observer.observe(document.body, { childList: true, subtree: true });
	}
	</script>
	'''
	#html += '</body></html>'

	# may add rust based minify-html here in future

	return Plot(type='concordance plot', html=html)

In [ ]:
conc_reuters.concordance_plot(token_str='cause', page_size=10, append_info=True).display()

In [ ]:
#| hide
conc_reuters.concordance_plot(token_str='love', page_size=10, append_info=True).display()

In [ ]:
#| hide
gardenparty = Corpus().load(path_to_gardenparty_corpus)
report_gardenparty = Concordance(gardenparty)

In [ ]:
#| hide
cp = report_gardenparty.concordance_plot('he')
cp.display()
cp.to_file('tmp.html')

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()